In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models_motif import LongRangeDNN
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
contacts='PP_PE'
path='/users/mtaranov/datasets_3d/dist_matched_'+contacts+'/'

X_train_node1 = path+'motifs/train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+'y_train_thres_10.npy')
y_test = get_labels(path+'y_test_thres_10.npy')
y_valid = get_labels(path+'y_valid_thres_10.npy')

X_train=concat_motifs([X_train_node1, X_train_node2, X_train_window])
X_test=concat_motifs([X_test_node1, X_test_node2, X_test_window])
X_valid=concat_motifs([X_valid_node1, X_valid_node2, X_valid_window])

In [10]:
X_train_pairs = X_train.reshape(X_train.shape[0],X_train.shape[2]*X_train.shape[3])
X_valid_pairs = X_valid.reshape(X_valid.shape[0],X_valid.shape[2]*X_valid.shape[3])
X_test_pairs = X_test.reshape(X_test.shape[0],X_test.shape[2]*X_test.shape[3])

In [11]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train_pairs, X_valid_pairs, X_test_pairs)

In [12]:
X_train_normalized.shape

(6130, 5760)

# from motifs

In [13]:
from models_motif import LongRangeDNN_FC
dnn_normalized = LongRangeDNN_FC(num_features=5760, use_deep_CNN=True)
validation_data = (X_valid_normalized, y_valid)
dnn_normalized.train(X_train_normalized, y_train, validation_data)

Training model...
Train on 6130 samples, validate on 3536 samples
Epoch 1/100
6000/6130 [============================>.] - ETA: 0s - loss: 2.7063Epoch 0: validation loss: 0.565
Balanced Accuracy: 74.18%	 auROC: 0.819	 auPRC: 0.827	 auPRG: 0.660
Recall at 5%|10%|20% FDR: 21.3%|34.3%|63.2%	 Num Positives: 1768	 Num Negatives: 1768	 

6130/6130 [==============================] - 2s - loss: 2.6693 - val_loss: 0.5652
Epoch 2/100
6000/6130 [============================>.] - ETA: 0s - loss: 0.5321Epoch 1: validation loss: 0.446
Balanced Accuracy: 79.78%	 auROC: 0.883	 auPRC: 0.887	 auPRG: 0.797
Recall at 5%|10%|20% FDR: 41.7%|60.5%|81.2%	 Num Positives: 1768	 Num Negatives: 1768	 

6130/6130 [==============================] - 2s - loss: 0.5324 - val_loss: 0.4458
Epoch 3/100
6000/6130 [============================>.] - ETA: 0s - loss: 0.2241Epoch 2: validation loss: 0.444
Balanced Accuracy: 80.88%	 auROC: 0.895	 auPRC: 0.896	 auPRG: 0.826
Recall at 5%|10%|20% FDR: 45.8%|69.3%|83.1%	 Num Positi

In [14]:
print(dnn_normalized.test(X_test_normalized, y_test))
print(dnn_normalized.test(X_train_normalized, y_train))
print(dnn_normalized.test(X_valid_normalized, y_valid))

Balanced Accuracy: 82.09%	 auROC: 0.901	 auPRC: 0.904	 auPRG: 0.837
Recall at 5%|10%|20% FDR: 51.6%|66.9%|85.2%	 Num Positives: 1189	 Num Negatives: 1189	 
Balanced Accuracy: 99.98%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 3065	 Num Negatives: 3065	 
Balanced Accuracy: 83.26%	 auROC: 0.908	 auPRC: 0.911	 auPRG: 0.854
Recall at 5%|10%|20% FDR: 56.1%|72.4%|87.0%	 Num Positives: 1768	 Num Negatives: 1768	 
